# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.5.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 2.8620, train_loss: 0.27537161, train_accuracy: 0.5312, test_Accuracy: 0.3502
Epoch: [ 0] [    1/  468] time: 3.8740, train_loss: 0.34401697, train_accuracy: 0.5234, test_Accuracy: 0.4880
Epoch: [ 0] [    2/  468] time: 4.8634, train_loss: 0.27018720, train_accuracy: 0.5938, test_Accuracy: 0.5739
Epoch: [ 0] [    3/  468] time: 5.8232, train_loss: 0.30082202, train_accuracy: 0.6797, test_Accuracy: 0.6101
Epoch: [ 0] [    4/  468] time: 6.8242, train_loss: 0.34786552, train_accuracy: 0.6953, test_Accuracy: 0.6344
Epoch: [ 0] [    5/  468] time: 7.7182, train_loss: 0.29289582, train_accuracy: 0.7031, test_Accuracy: 0.6690
Epoch: [ 0] [    6/  468] time: 8.6012, train_loss: 0.27103245, train_accuracy: 0.6875, test_Accuracy: 0.6954
Epoch: [ 0] [    7/  468] time: 9.5582, train_loss: 0.27577981, train_accuracy: 0.7344, test_Accuracy: 0.7141
Epoch: [ 0] [    8/  468] time: 10.4733

Epoch: [ 0] [   74/  468] time: 70.9623, train_loss: 0.07822019, train_accuracy: 0.9609, test_Accuracy: 0.9227
Epoch: [ 0] [   75/  468] time: 71.8633, train_loss: 0.15872654, train_accuracy: 0.9297, test_Accuracy: 0.9263
Epoch: [ 0] [   76/  468] time: 72.7673, train_loss: 0.09403252, train_accuracy: 0.9766, test_Accuracy: 0.9291
Epoch: [ 0] [   77/  468] time: 73.6963, train_loss: 0.12394293, train_accuracy: 0.9453, test_Accuracy: 0.9321
Epoch: [ 0] [   78/  468] time: 74.6493, train_loss: 0.25185615, train_accuracy: 0.9062, test_Accuracy: 0.9344
Epoch: [ 0] [   79/  468] time: 75.5424, train_loss: 0.16334257, train_accuracy: 0.9297, test_Accuracy: 0.9369
Epoch: [ 0] [   80/  468] time: 76.4423, train_loss: 0.07922050, train_accuracy: 0.9766, test_Accuracy: 0.9377
Epoch: [ 0] [   81/  468] time: 77.3075, train_loss: 0.15742485, train_accuracy: 0.9609, test_Accuracy: 0.9388
Epoch: [ 0] [   82/  468] time: 78.2463, train_loss: 0.10561615, train_accuracy: 0.9453, test_Accuracy: 0.9396
E

Epoch: [ 0] [  148/  468] time: 135.0223, train_loss: 0.09208603, train_accuracy: 0.9766, test_Accuracy: 0.9563
Epoch: [ 0] [  149/  468] time: 135.8293, train_loss: 0.14874858, train_accuracy: 0.9531, test_Accuracy: 0.9562
Epoch: [ 0] [  150/  468] time: 136.5744, train_loss: 0.09304371, train_accuracy: 0.9609, test_Accuracy: 0.9560
Epoch: [ 0] [  151/  468] time: 137.3163, train_loss: 0.13258657, train_accuracy: 0.9453, test_Accuracy: 0.9560
Epoch: [ 0] [  152/  468] time: 138.0483, train_loss: 0.16539337, train_accuracy: 0.9453, test_Accuracy: 0.9558
Epoch: [ 0] [  153/  468] time: 138.7403, train_loss: 0.10524774, train_accuracy: 0.9688, test_Accuracy: 0.9560
Epoch: [ 0] [  154/  468] time: 139.5113, train_loss: 0.11404105, train_accuracy: 0.9766, test_Accuracy: 0.9554
Epoch: [ 0] [  155/  468] time: 140.2373, train_loss: 0.09369515, train_accuracy: 0.9688, test_Accuracy: 0.9557
Epoch: [ 0] [  156/  468] time: 140.9523, train_loss: 0.06039258, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 189.6821, train_loss: 0.14285710, train_accuracy: 0.9688, test_Accuracy: 0.9575
Epoch: [ 0] [  223/  468] time: 190.1750, train_loss: 0.10288278, train_accuracy: 0.9766, test_Accuracy: 0.9590
Epoch: [ 0] [  224/  468] time: 190.6980, train_loss: 0.09679151, train_accuracy: 0.9609, test_Accuracy: 0.9603
Epoch: [ 0] [  225/  468] time: 191.2430, train_loss: 0.09821682, train_accuracy: 0.9688, test_Accuracy: 0.9617
Epoch: [ 0] [  226/  468] time: 191.7140, train_loss: 0.12260213, train_accuracy: 0.9688, test_Accuracy: 0.9613
Epoch: [ 0] [  227/  468] time: 192.2260, train_loss: 0.03272253, train_accuracy: 0.9922, test_Accuracy: 0.9601
Epoch: [ 0] [  228/  468] time: 192.9660, train_loss: 0.08307144, train_accuracy: 0.9688, test_Accuracy: 0.9592
Epoch: [ 0] [  229/  468] time: 193.7671, train_loss: 0.14641729, train_accuracy: 0.9531, test_Accuracy: 0.9594
Epoch: [ 0] [  230/  468] time: 194.5211, train_loss: 0.04234621, train_accuracy: 1.0000, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 244.3805, train_loss: 0.06681886, train_accuracy: 0.9688, test_Accuracy: 0.9598
Epoch: [ 0] [  297/  468] time: 245.1610, train_loss: 0.13260171, train_accuracy: 0.9688, test_Accuracy: 0.9599
Epoch: [ 0] [  298/  468] time: 245.9450, train_loss: 0.07641497, train_accuracy: 0.9531, test_Accuracy: 0.9605
Epoch: [ 0] [  299/  468] time: 246.6971, train_loss: 0.06312172, train_accuracy: 0.9844, test_Accuracy: 0.9603
Epoch: [ 0] [  300/  468] time: 247.4770, train_loss: 0.10858423, train_accuracy: 0.9766, test_Accuracy: 0.9603
Epoch: [ 0] [  301/  468] time: 248.1900, train_loss: 0.10263545, train_accuracy: 0.9219, test_Accuracy: 0.9600
Epoch: [ 0] [  302/  468] time: 248.9960, train_loss: 0.04709834, train_accuracy: 0.9609, test_Accuracy: 0.9601
Epoch: [ 0] [  303/  468] time: 249.7480, train_loss: 0.04987021, train_accuracy: 0.9922, test_Accuracy: 0.9606
Epoch: [ 0] [  304/  468] time: 250.4860, train_loss: 0.09253675, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 300.5381, train_loss: 0.05318908, train_accuracy: 0.9766, test_Accuracy: 0.9692
Epoch: [ 0] [  371/  468] time: 301.3151, train_loss: 0.07588097, train_accuracy: 0.9688, test_Accuracy: 0.9700
Epoch: [ 0] [  372/  468] time: 302.0831, train_loss: 0.05537463, train_accuracy: 0.9922, test_Accuracy: 0.9692
Epoch: [ 0] [  373/  468] time: 302.9104, train_loss: 0.07047355, train_accuracy: 0.9766, test_Accuracy: 0.9695
Epoch: [ 0] [  374/  468] time: 303.7120, train_loss: 0.08562930, train_accuracy: 0.9688, test_Accuracy: 0.9701
Epoch: [ 0] [  375/  468] time: 304.4630, train_loss: 0.07618178, train_accuracy: 0.9766, test_Accuracy: 0.9704
Epoch: [ 0] [  376/  468] time: 305.2440, train_loss: 0.07545616, train_accuracy: 0.9766, test_Accuracy: 0.9706
Epoch: [ 0] [  377/  468] time: 306.0170, train_loss: 0.10796739, train_accuracy: 0.9609, test_Accuracy: 0.9703
Epoch: [ 0] [  378/  468] time: 306.7301, train_loss: 0.03310665, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 355.4311, train_loss: 0.10404128, train_accuracy: 0.9766, test_Accuracy: 0.9692
Epoch: [ 0] [  445/  468] time: 356.1300, train_loss: 0.05535939, train_accuracy: 0.9766, test_Accuracy: 0.9689
Epoch: [ 0] [  446/  468] time: 356.8461, train_loss: 0.06696216, train_accuracy: 0.9844, test_Accuracy: 0.9700
Epoch: [ 0] [  447/  468] time: 357.6070, train_loss: 0.05049480, train_accuracy: 0.9844, test_Accuracy: 0.9699
Epoch: [ 0] [  448/  468] time: 358.3520, train_loss: 0.04898415, train_accuracy: 0.9688, test_Accuracy: 0.9702
Epoch: [ 0] [  449/  468] time: 359.1020, train_loss: 0.04497945, train_accuracy: 0.9844, test_Accuracy: 0.9714
Epoch: [ 0] [  450/  468] time: 359.8281, train_loss: 0.06713687, train_accuracy: 0.9688, test_Accuracy: 0.9710
Epoch: [ 0] [  451/  468] time: 360.6420, train_loss: 0.09948654, train_accuracy: 0.9766, test_Accuracy: 0.9713
Epoch: [ 0] [  452/  468] time: 361.4150, train_loss: 0.05150383, train_accuracy: 0.9922, test_Accuracy:

# Test accuracy : 96.59 %